In [8]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import os 
import time
import numpy

def get_income_statement(number, year, season):
    
    url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb04'
    payload = {
                'encodeURIComponent': '1',
                'step': '1',
                'firstin': '1',
                'off': '1',
                'queryName': 'co_id',
                'inpuType': 'co_id',
                'TYPEK': 'all',
                'isnew': 'false',
                'co_id': str(number),
                'year': str(year),
                'season': str(season),
            }
    
    payload_1={
        'encodeURIComponent': '1',
        'TYPEK': 'sii',
        'step': '2',
        'year': str(year),
        'season': str(season),
        'co_id': str(number),
        'firstin': '1 '
          }

    try:           
        res = requests.post(url , data = payload)
        df = pd.read_html(res.text)[1]
        soup = BeautifulSoup(res.text , 'html.parser')
    except IndexError :
        res = requests.post(url , data = payload_1)
        df = pd.read_html(res.text)[1]
        soup = BeautifulSoup(res.text , 'html.parser')
                
    
    name=soup.find('h4',align="center").text #中文名稱
    name1=name[:4]
    name2=name[-4:]
    name=name.strip(name1).strip(name2)

    
    str_1='基本每股盈餘'
    str_2='營業收入合計'
    str_3='營業毛利（毛損）'
    str_4='營業外收入及支出合計'


    for i in range(0,100): #基本每股盈餘
        index=0
        try:
            if df.iat[i,0] == str_1:
                index=i
                EPS=df.iat[index+1,1]
                break
        except IndexError:
            EPS=0
            

    for i in range(0,100): #營業收入合計
        try:
            if df.iat[i,0] == str_2:
                index=i
                income=df.iat[index,1]   
                break
        except IndexError:
                income=0 #金融股 無income

    for i in range(0,100): #營業毛利（毛損）
        index=0
        try:
            if df.iat[i,0] == str_3:
                index=i
                Gross_Profit=df.iat[index,1]
                break
        except IndexError:
                Gross_Profit=0 #無Gross_Profit

    for i in range(0,100): #營業外收入及支出合計
        index=0
        try:
            if df.iat[i,0] == str_4:
                index=i
                non_operating_income_and_expenses=df.iat[index,1]
                break
        except IndexError:
                non_operating_income_and_expenses=0 #無non_operating_income_and_expenses
    
    
    quarter=f'民國{year}第{season}季'

    return name,income,non_operating_income_and_expenses,Gross_Profit,EPS,quarter

def consolidated_balance_sheet(number,year,season):
    url='https://mops.twse.com.tw/mops/web/ajax_t164sb03'

    payload={
            'encodeURIComponent': '1',
            'step': '1',
            'firstin':'1' ,
            'off': '1',
            'queryName': 'co_id',
            'inpuType': 'co_id',
            'TYPEK': 'all',
            'isnew': 'false',
            'co_id': str(number),
            'year': str(year),
            'season': str(season)
        }    
        
    payload_1={
        'encodeURIComponent': '1',
        'TYPEK': 'sii',
        'step': '2',
        'year': str(year),
        'season': str(season),
        'co_id': str(number),
        'firstin': '1'
    }
                    
    try:
        res = requests.post(url , data = payload)
        df = pd.read_html(res.text)[1]
    except IndexError:  
        res = requests.post(url , data = payload_1)
        df = pd.read_html(res.text)[1]
        

    str_5='存貨'  
    for i in range(0,100): #存貨
        try:
            if df.iat[i,0] == str_5:
                index=i
                stock=df.iat[index,1]
                break
        except IndexError:
            stock=0 #無存貨
    
    return stock

if __name__ == "__main__" :
    
    box_name=[]
    box_number=[]
    box_quarter=[]
    box_income=[]
    box_Gross_Profit=[]
    box_EPS=[]
    box_non_operating_income_and_expenses=[]
    box_stock=[]
    count=1
    
    df=pd.DataFrame(columns=['股票代碼','股票名稱','季度','營業收入合計','營業外收入及支出合計','營業毛利','基本每股盈餘','存貨'])
    
    number=int(input("Please enter the stock number:(ex:2330、2412)"))
    
    start_year=int(input("Please enter the start year:(ex:099、100)"))
    start_season=int(input("Please enter the start season:(ex:1、2、3、4)"))
    end_year=int(input("Please enter the end year:(ex:099、111)"))
    end_season=int(input("Please enter the end season:(ex:1、2、3、4)"))
    save_path=str(input("Please enter the save_path:(ex:./2330)"))
    
    for y in range(start_year,end_year+1):
        time.sleep(5)
        for s in range(1,5):
                time.sleep(2)
                try:
                    stock=consolidated_balance_sheet(number, y, s)
                    box_stock.append(stock)     
                except ValueError:   
                    break

                name,income,non_operating_income_and_expenses,Gross_Profit,EPS,quarter=get_income_statement(number, y, s)
                box_name.append(name)
                box_quarter.append(quarter)
                box_income.append(income)
                box_Gross_Profit.append(Gross_Profit)
                box_EPS.append(EPS)
                box_non_operating_income_and_expenses.append(non_operating_income_and_expenses)
                count=count+1

    for c in  range(1,count):
        box_number.append(number)
        

df['股票代碼']=box_number
df['股票名稱']=box_name
df['存貨']=box_stock
df['季度']=box_quarter
df['營業收入合計']=box_income #營業收入合計
df['營業外收入及支出合計']=box_non_operating_income_and_expenses #營業外收入及支出合計
df['營業毛利']=box_Gross_Profit #營業毛利
df['基本每股盈餘']=box_EPS #基本每股盈餘  

if start_season !=1:
    dele_head=start_season-1
    df=df.drop(labels=range(0,dele_head))

if end_season == 2 | end_season==3:
    df=df.drop(labels=range((count-1)-(4-end_season),count-1))


if not os.path.exists(save_path):
    os.makedirs(save_path)

df.to_excel(save_path+f'./{number}_{name}_from_{start_year}_{start_season}_to_{end_year}_{end_season}.xlsx',encoding='utf-8')

Please enter the stock number:(ex:2330、2412)2881
Please enter the start year:(ex:099、100)104
Please enter the start season:(ex:1、2、3、4)4
Please enter the end year:(ex:099、111)111
Please enter the end season:(ex:1、2、3、4)1
Please enter the save_path:(ex:./2330)./2881


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))